## This notebook check the M2 state transitions and force commands

In [ ]:
from lsst.ts import salobj
import asyncio
import os

import numpy as np
from matplotlib import pyplot as plt
from astropy.time import Time
from datetime import datetime, timedelta
import pandas as pd

from lsst.ts.idl.enums import MTM2

In [ ]:
summit = 1 #use this for summit testing
#summit = 0 #use this for NCSA

In [ ]:
if summit:
    os.environ["OSPL_URI"]="file:///home/bxin/WORK/ts_ddsconfig/config/ospl-sp.xml"
    os.environ["LSST_DDS_DOMAIN_ID"] = "11"
    print(os.environ["OSPL_URI"])
    print(os.environ["LSST_DDS_PARTITION_PREFIX"])
    print(os.environ["LSST_DDS_DOMAIN_ID"])
else:
    import os
    print(os.environ["OSPL_URI"])
    if os.environ.get("LSST_DDS_ALIGNER", "false") != "false":
        print("LSST_DDS_ALIGNER is mis-configured")

In [ ]:
start_time = datetime.now()
script = salobj.Controller("Script", index=1)
if not summit:
    await asyncio.sleep(10) #no need to wait on summit since we use the daemon
    ptg = salobj.Remote(script.domain, "MTPtg")
    rot = salobj.Remote(script.domain, "MTRotator")
mount = salobj.Remote(script.domain, "MTMount") #we still need it on the summit, to check (not generate) fake mount telemetry!
m2 = salobj.Remote(script.domain, "MTM2", exclude=['logMessage'])
print(f'time to start is {datetime.now() - start_time} [s]')

In [ ]:
if summit:
    await asyncio.gather(m2.start_task,
                         mount.start_task)
else:
    await asyncio.gather(ptg.start_task,
                         mount.start_task,
                         script.start_task, #                     
                         rot.start_task,
                         m2.start_task)

In [ ]:
await m2.evt_heartbeat.next(flush=True, timeout=5)

### test state transition. If m2 is already enabled, disable then enable it.

In [ ]:
if summit:
    await m2.cmd_enterControl.set_start() 

In [ ]:
sstate = await m2.evt_summaryState.aget(timeout=5)
print('staring with: m2 state',salobj.State(sstate.summaryState), pd.to_datetime(sstate.private_sndStamp, unit='s'))
if summit and sstate.summaryState == salobj.State.OFFLINE:
    #only available in offline on the summit
    dstate = await m2.evt_detailedState.aget(timeout=5)
    print('m2 detailed state', dstate.detailedState, pd.to_datetime(dstate.private_sndStamp, unit='s'))
    #print('m2 state', MTM2.DetailedState(dstate.detailedState), pd.to_datetime(dstate.private_sndStamp, unit='s'))

In [ ]:
sstate = await m2.evt_summaryState.aget(timeout=5)
print('staring with: m2 state',salobj.State(sstate.summaryState), pd.to_datetime(sstate.private_sndStamp, unit='s'))

In [ ]:
#enums is not yet implemented.

#Te-Wei:
#publishOnly = 1
#Available = 2
#When you see 2, this means the M2 is DDS contrallable.

#for i in range(1,3):
#    print(i, MTM2.DetailedState(i))

Depending on what state M2 is in, we need to use the TWO cells below repeatedly to get to the state we want

In [ ]:
#execute once to get the state OUT one level
if sstate.summaryState == salobj.State.ENABLED:
    await m2.cmd_disable.set_start()
if sstate.summaryState == salobj.State.DISABLED:
    await m2.cmd_standby.set_start()
#the below would shut down the CSC; be ready to restart it if you want to try.
if sstate.summaryState == salobj.State.STANDBY:
    await m2.cmd_exitControl.set_start()
sstate = await m2.evt_summaryState.next(flush=True, timeout=5)
print('starting with: m2 state',salobj.State(sstate.summaryState), pd.to_datetime(sstate.private_sndStamp, unit='s'))

In [ ]:
#execute once to get the state UP one level
if sstate.summaryState == salobj.State.FAULT:
    await m2.cmd_clearErrors.set_start()
if sstate.summaryState == salobj.State.OFFLINE:
    await m2.cmd_enterControl.set_start()    
if sstate.summaryState == salobj.State.STANDBY:
    await m2.cmd_start.set_start() #go to DISABLE
if sstate.summaryState == salobj.State.DISABLED:
    await m2.cmd_enable.set_start() #go to ENABLE = closed loop
sstate = await m2.evt_summaryState.next(flush=True, timeout=5) 
print('starting with: m2 state',salobj.State(sstate.summaryState), pd.to_datetime(sstate.private_sndStamp, unit='s'))

Check that EFD has the state transition events

In [ ]:
from lsst_efd_client import EfdClient

if summit:
    client = EfdClient('summit_efd')
else:
    client = EfdClient('ncsa_teststand_efd')
csc_index = 1
while True:
    end = Time(datetime.now(), scale='tai')
    start = end - timedelta(hours=1)
    dfe = await client.select_time_series('lsst.sal.MTM2.logevent_summaryState', '*', start, end, csc_index)
    #print(start, end, len(dfe))
    if len(dfe)>0:
        break
#to check messages in Kafka, go to https://lsst-kafka-0-nts-efd.ncsa.illinois.edu/
##we cannot get time series data from DDS. We have to query the EFD

In [ ]:
dfe

### check the forces

In [ ]:
m2ForceBalance = await m2.evt_forceBalanceSystemStatus.aget(timeout=10.)
print("starting with Status of the M2 force balance system ---", m2ForceBalance.status, "----",
      pd.to_datetime(m2ForceBalance.private_sndStamp, unit='s'))
if not m2ForceBalance.status:
    await m2.cmd_switchForceBalanceSystem.set_start(status=True, timeout=10)
    m2ForceBalance = await m2.evt_forceBalanceSystemStatus.aget(timeout=10.)
    print("Status of the M2 force balance system", m2ForceBalance.status)

In [ ]:
fa = [0]*72
ft = [0]*6
await m2.cmd_applyForces.set_start(axial=fa, tangent=ft)

In [ ]:
await m2.cmd_resetForceOffsets.set_start()

In [ ]:
axialForces = await m2.tel_axialForce.aget(timeout=2)
tangentForces = await m2.tel_tangentForce.aget(timeout=2)

In [ ]:
def plotForces(axialForces, tangentForces):
    fig, ax = plt.subplots(2,1, figsize=(15,8))
    ax[0].plot(axialForces.measured, label='measured');
    ax[0].plot(axialForces.applied, label='applied');
    ax[0].plot(axialForces.hardpointCorrection,'.', label='FB');
    ax[0].plot(axialForces.lutGravity, label='LUT G');
    ax[0].legend()
    ax[1].plot(tangentForces.measured, label='measured');
    ax[1].plot(tangentForces.applied, label='applied');
    ax[1].plot(tangentForces.hardpointCorrection, 'o', label='FB');
    ax[1].plot(tangentForces.lutGravity, label='LUT G');
    ax[1].legend()

    aa = np.loadtxt('%s/notebooks/M2_FEA/data/M2_1um_72_force.txt'%(os.environ["HOME"]))
    # to have +x going to right, and +y going up, we need to transpose and reverse x and y
    xact = -aa[:,2]
    yact = -aa[:,1]

    fig2, ax=plt.subplots( 1,2, figsize = [10,4])
    aa = np.array(axialForces.measured)
    img = ax[0].scatter(xact, yact, c=aa, s=abs(aa)*2)
    #plt.jet()
    ax[0].axis('equal')
    ax[0].set_title('measured forces')
    fig.colorbar(img, ax=ax[0])

    aa = np.array(axialForces.applied)
    img = ax[1].scatter(xact, yact, c=aa, s=abs(aa)*2)
    #plt.jet()
    ax[1].axis('equal')
    ax[1].set_title('applied forces')
    fig.colorbar(img, ax=ax[1])

In [ ]:
plotForces(axialForces, tangentForces)

In [ ]:
fa = [2]*72
ft = [-2]*6
await m2.cmd_applyForces.set_start(axial=fa, tangent=ft)

In [ ]:
axialForces = await m2.tel_axialForce.aget(timeout=2)
tangentForces = await m2.tel_tangentForce.aget(timeout=2)
plotForces(axialForces, tangentForces)

In [ ]:
zAngle = await m2.tel_zenithAngle.aget(timeout=5)
print('zenith angle = ', zAngle.measured)

### Close up

In [ ]:
#if we started with enabled state, we need to put it back
await salobj.set_summary_state(m2, salobj.State.ENABLED) #enable m2